# Simulation Setup
In this section, we first outline the simulation setup for our comparison run.

## Prepare the Dengo-Template Files
To incorporate $\texttt{Dengo}$ to the current $\texttt{Enzo}$ implementation, we have created a helper function that helps generate the files required with $\texttt{Dengo}$/$\texttt{Jinja2}$. The github repo can be found here https://github.com/hisunnytang/dengo-templates. By placing the newly generated source files to the `enzo/src/enzo`, the compilation of the `enzo` executable can be done with the switch `make dengo-yes`. For more details, please read the `README.md` in the above mentioned repo.

## MUSIC

$\texttt{Enzo}$ does not generate cosmological initial conditions. It relies on external packages to do so. And a detailed description on how to generate various initial conditions are documented in [Enzo User Guide](https://enzo.readthedocs.io/en/latest/user_guide/CosmologicalInitialConditions.html).
$\texttt{MUSIC}$ [Hahn and Abel (2011)](http://arxiv.org/abs/1103.6031) can generate "uniform" and "zoomed" initial condition files.

Below is the music configuration file. The simulation box is initialized a width of $0.3 \mathrm{Mpc/h}$ at $z = 100$. The `ref_center` refers to the reference center of the simulation. Our simulation is centered on a $10^6 M_\odot$ dark matter halo based on a coarser initial dark matter only runs. It is a multi-resolution grid with the base resolution of 128 and a highest effective resolution of 512. The initial conditions can be generated with the command
```bash
./MUSIC music_input.conf
```
The initial baryon densities, velocities as well as dark matter particle displacements and velocities are placed under the directory `bechem_z100_baryon`. A (incomplete) enzo parameter file `parameter_file.txt` is also generated. This specified the grid specification for enzo.

```python
[setup]
boxlength  = 0.3
zstart     = 100
levelmin   = 7
levelmin_TF = 7
levelmax   = 9
ref_center = 0.10518512, 0.89903345, 0.39035773
ref_extent = 0.2, 0.2, 0.2
padding    = 8
overlap    = 4
align_top  = yes
baryons    = yes
use_2LPT   = no
use_LLA    = no
periodic_TF = yes
[random]
seed[7]    = 23699
seed[8]    = 95602
seed[9]    = 39181
seed[10]   = 45678
seed[11]   = 56789
seed[12]   = 67890
[cosmology]
Omega_m    = 0.276
Omega_L    = 0.724
w0         = -1.0
Omega_b    = 0.045
H0         = 70.3
sigma_8    = 0.811
nspec      = 0.961
transfer   = eisenstein
[output]
format     = enzo
filename = bechem_z100_baryon
```

## Enzo Setup
This outlines the input parameter file to `Enzo`. It is mostly related to the refinement schemes and how to specify chemistry related inputs. 


### Refinement Schemes
- `MaximumRefinementLevel` is set to 31 which converts to a maximum resolution of $\sim 0.02 AU$
- The cell is refined with flags `2 4 6 7 8` [Enzo Parameter List](https://enzo.readthedocs.io/en/latest/parameters/index.html)
- `2 4` refers to refinement over baryon mass and particle mass
- `6` refers to refinement by Jeans length, we require Jeans Length to be resolved by 64 cells, with a refinement cold temperature of $200 \mathrm{K}$.
- `7` Refine if (cooling time < cell width/sound speed)
- `8` Refine by must-refine particles

```python
# grid refinement parameters
StaticHierarchy                       = 0
MaximumRefinementLevel                = 31
MaximumGravityRefinementLevel         = 31
MaximumParticleRefinementLevel        = 9
RefineBy                              = 2
CellFlaggingMethod                    = 2 4 6 7 8
MinimumOverDensityForRefinement       = 0.015625 0.015625 0 0 0
MinimumMassForRefinementLevelExponent = -0.3 -0.3 0 0 0
RefineByJeansLengthSafetyFactor    = 64
JeansRefinementColdTemperature     = 200.0
```

### Dengo Chemistry-Related Parameters

We outline the dengo related flags.
To use dengo, grackle has to be deactivated first `use_grackle = 0` and user has to set `use_dengo = 1`.
`RadiativeCooling = 1` enables thermal energy to be updated. `MultiSpecies = 0` is such that Grackle related flags are not activated.
`UseCoolingTimestep = 1` forces the minimum timestep has to be always smaller than 0.1 times the thermal timescale.
`dengo_reltol` specifies the 
Unlike `Grackle` where the initial abundance for various species are calculated internally, we can specify the relative abundance with `CosmologySimulation{species.name}Fraction`.
To maintain the consistency with the Grackle simulations, these are the values taken from the internal Grackle setup

```python
RadiativeCooling        = 1
use_grackle             = 0
use_dengo = 1
MultiSpecies            = 0 // sets grackle.primordial_chemistry
MetalCooling            = 0 // sets grackle.metal_cooling
grackle_data_file       = CloudyData_UVB=HM2012.h5
UseCoolingTimestep = 1
dengo_reltol = 1e-04


CosmologySimulationH2_1Fraction = 6.091980372629213e-07
CosmologySimulationH2_2Fraction = 6.533362733112322e-14
CosmologySimulationH_1Fraction = 0.7599993907786267
CosmologySimulationH_2Fraction = 0.0001514542764576715
CosmologySimulationH_m0Fraction = 2.3270797113461643e-11
CosmologySimulationHe_1Fraction = 0.2399999999999904
CosmologySimulationHe_2Fraction = 9.6e-15
CosmologySimulationHe_3Fraction = 9.6e-18
CosmologySimulationdeFraction = 0.00015145425322194602
```